In [6]:
import pandas as pd
import geopandas as gpd
import xarray as xr
import numpy as np

# Charger les fichiers de données
chirps_path = '/home/laye/Documents/GUIGMA/CHIRPS/merge_chirps-v2.0_1981_2024_p25_Senegal.nc'
shapefile_path = '/home/laye/Documents/GUIGMA/CHIRPS/gadm36_SEN_1.shp'

chirps_data = xr.open_dataset(chirps_path)
regions = gpd.read_file(shapefile_path)

# Indices de vulnérabilité et de capacité d'adaptation
vulnerability_cc_data = {
    'Region': ['Dakar', 'Diourbel', 'Fatick', 'Kaffrine', 'Kaolack', 'Kédougou', 'Kolda', 'Louga', 'Matam', 'Saint-Louis', 'Sédhiou', 'Tambacounda', 'Thiès', 'Ziguinchor'],
    'Vulnerability': [0.9, 1.6, 1.4, 1.4, 1.2, 3.1, 1.5, 3.1, 4.3, 1.2, 1.2, 2.2, 1.3, 0.9],
    'Lack of Coping Capacity': [3.7, 4.3, 5.6, 5.1, 4.6, 6.6, 6.3, 4.6, 5.4, 4.9, 6, 6, 4.3, 5.5]
}

vulnerability_cc_df = pd.DataFrame(vulnerability_cc_data)

results = []

# Liste des années pour lesquelles nous allons extraire les données 
years = list(range(1981, 2023))

# Itérer sur chaque région
for region_name in regions['NAME_1'].unique():
    region_shape = regions[regions['NAME_1'] == region_name]
    
    # Masquer les données CHIRPS en utilisant la région
    chirps_region = chirps_data.sel(latitude=slice(region_shape.bounds.miny.values[0], region_shape.bounds.maxy.values[0]), 
                                    longitude=slice(region_shape.bounds.minx.values[0], region_shape.bounds.maxx.values[0]))
    
    annual_precipitations = []
    dry_days_list = []

    for year in years:
        # Extraire les données pour l'année spécifiée
        chirps_region_year = chirps_region.sel(time=str(year))
        
        # Calculer la précipitation annuelle moyenne
        annual_precipitation = chirps_region_year['precip'].sum(dim='time').mean(dim=['latitude', 'longitude']).values
        
        # Calculer la précipitation mensuelle
        monthly_precipitation = chirps_region_year['precip'].groupby('time.month').sum(dim='time').mean(dim=['latitude', 'longitude'])
        
        # Identifier les 3 mois les plus pluvieux
        top3_rainy_months = monthly_precipitation.sortby(monthly_precipitation, ascending=False).month[:3].values
        
        # Extraire les données pour les 3 mois les plus pluvieux
        rainy_months_data = chirps_region_year.sel(time=chirps_region_year['time.month'].isin(top3_rainy_months))
        
        # Calculer le nombre de jours sans précipitations (<1 mm)
        dry_days = (rainy_months_data['precip'] < 1).sum(dim='time').mean(dim=['latitude', 'longitude']).values
        
        # Ajouter aux listes
        
        annual_precipitations.append(annual_precipitation)
        dry_days_list.append(dry_days)
    
    # Calculer la moyenne des précipitations annuelles et des jours secs
    average_precipitation = np.mean(annual_precipitations)
    average_dry_days = np.mean(dry_days_list)

    # Ajouter les résultats à la liste
    results.append([region_name, average_precipitation.item(), average_dry_days.item()])

# Convertir les résultats en DataFrame
hazard_df = pd.DataFrame(results, columns=['Region', 'Average Annual Precipitation (mm)', 'Average Dry Days (<1 mm)'])

# Fusionner les données de hazard avec les indices de vulnérabilité et de capacité d'adaptation
merged_df = pd.merge(hazard_df, vulnerability_cc_df, left_on='Region', right_on='Region')

# Sauvegarder les résultats dans un fichier CSV
output_path = '/home/laye/Documents/GUIGMA/dry_spell/RISK_hazard_vulnerability_capacity.csv'
merged_df.to_csv(output_path, index=False)

# Afficher les résultats
print(merged_df)


         Region  Average Annual Precipitation (mm)  Average Dry Days (<1 mm)  \
0         Dakar                         211.471681                 27.000000   
1      Diourbel                         455.699671                 53.004762   
2        Fatick                         589.632289                 46.755238   
3      Kaffrine                         541.085458                 49.052381   
4       Kaolack                         594.642467                 45.379464   
5      Kédougou                        1039.506033                 33.698214   
6         Kolda                         871.919257                 37.893849   
7         Louga                         360.298459                 57.030234   
8         Matam                         400.650949                 57.435374   
9   Saint-Louis                         287.529878                 64.102092   
10      Sédhiou                         931.287531                 34.815476   
11  Tambacounda                         

In [8]:
import pandas as pd
import geopandas as gpd
import xarray as xr
import numpy as np

# Charger les fichiers de données
chirps_path = '/home/laye/Documents/GUIGMA/CHIRPS/merge_chirps-v2.0_1981_2024_p25_Senegal.nc'
shapefile_path = '/home/laye/Documents/GUIGMA/CHIRPS/gadm36_SEN_1.shp'

chirps_data = xr.open_dataset(chirps_path)
regions = gpd.read_file(shapefile_path)

# Indices de vulnérabilité et de capacité d'adaptation
vulnerability_cc_data = {
    'Region': ['Dakar', 'Diourbel', 'Fatick', 'Kaffrine', 'Kaolack', 'Kédougou', 'Kolda', 'Louga', 'Matam', 'Saint-Louis', 'Sédhiou', 'Tambacounda', 'Thiès', 'Ziguinchor'],
    'Vulnerability': [0.9, 1.6, 1.4, 1.4, 1.2, 3.1, 1.5, 3.1, 4.3, 1.2, 1.2, 2.2, 1.3, 0.9],
    'Lack of Coping Capacity': [3.7, 4.3, 5.6, 5.1, 4.6, 6.6, 6.3, 4.6, 5.4, 4.9, 6, 6, 4.3, 5.5]
}

vulnerability_cc_df = pd.DataFrame(vulnerability_cc_data)

results = []

# Liste des années pour lesquelles nous allons extraire les données 
years = list(range(1981, 2023))

# Fonction pour effectuer le rescaling entre 10 et 100
def rescale_to_100(values):
    return (values - np.min(values)) / (np.max(values) - np.min(values)) * 90 + 10

# Itérer sur chaque région
for region_name in regions['NAME_1'].unique():
    region_shape = regions[regions['NAME_1'] == region_name]
    
    # Masquer les données CHIRPS en utilisant la région
    chirps_region = chirps_data.sel(latitude=slice(region_shape.bounds.miny.values[0], region_shape.bounds.maxy.values[0]), 
                                    longitude=slice(region_shape.bounds.minx.values[0], region_shape.bounds.maxx.values[0]))
    
    annual_precipitations = []
    dry_days_list = []

    for year in years:
        # Extraire les données pour l'année spécifiée
        chirps_region_year = chirps_region.sel(time=str(year))
        
        # Calculer la précipitation annuelle moyenne
        annual_precipitation = chirps_region_year['precip'].sum(dim='time').mean(dim=['latitude', 'longitude']).values
        
        # Calculer la précipitation mensuelle
        monthly_precipitation = chirps_region_year['precip'].groupby('time.month').sum(dim='time').mean(dim=['latitude', 'longitude'])
        
        # Identifier les 3 mois les plus pluvieux
        top3_rainy_months = monthly_precipitation.sortby(monthly_precipitation, ascending=False).month[:3].values
        
        # Extraire les données pour les 3 mois les plus pluvieux
        rainy_months_data = chirps_region_year.sel(time=chirps_region_year['time.month'].isin(top3_rainy_months))
        
        # Calculer le nombre de jours sans précipitations (<1 mm)
        dry_days = (rainy_months_data['precip'] < 1).sum(dim='time').mean(dim=['latitude', 'longitude']).values
        
        # Ajouter aux listes
        annual_precipitations.append(annual_precipitation)
        dry_days_list.append(dry_days)
    
    # Calculer la moyenne des précipitations annuelles et des jours secs
    average_precipitation = np.mean(annual_precipitations)
    average_dry_days = np.mean(dry_days_list)

    # Ajouter les résultats à la liste
    results.append([region_name, average_precipitation.item(), average_dry_days.item()])

# Convertir les résultats en DataFrame
hazard_df = pd.DataFrame(results, columns=['Region', 'Average Annual Precipitation (mm)', 'Average Dry Days (<1 mm)'])

# Rescaling des 4 variables : Précipitation annuelle, Jours secs, Vulnérabilité, Capacité d'adaptation
hazard_df['Average Annual Precipitation (mm) Rescaled'] = rescale_to_100(hazard_df['Average Annual Precipitation (mm)'])
hazard_df['Average Dry Days (<1 mm) Rescaled'] = rescale_to_100(hazard_df['Average Dry Days (<1 mm)'])

# Fusionner les données de hazard avec les indices de vulnérabilité et de capacité d'adaptation
merged_df = pd.merge(hazard_df, vulnerability_cc_df, left_on='Region', right_on='Region')

# Rescaling pour 'Vulnerability' et 'Lack of Coping Capacity'
merged_df['Vulnerability Rescaled'] = rescale_to_100(merged_df['Vulnerability'])
merged_df['Lack of Coping Capacity Rescaled'] = rescale_to_100(merged_df['Lack of Coping Capacity'])

# Créer un DataFrame final contenant seulement les variables rééchelonnées
final_rescaled_df = merged_df[['Region', 
                               'Average Annual Precipitation (mm) Rescaled', 
                               'Average Dry Days (<1 mm) Rescaled', 
                               'Vulnerability Rescaled', 
                               'Lack of Coping Capacity Rescaled']]

# Sauvegarder les résultats rééchelonnés dans un fichier CSV
output_path = '/home/laye/Documents/GUIGMA/dry_spell/RISK_rescaled_variables.csv'
final_rescaled_df.to_csv(output_path, index=False)

# Afficher les résultats
print(final_rescaled_df)


         Region  Average Annual Precipitation (mm) Rescaled  \
0         Dakar                                   10.000000   
1      Diourbel                                   32.871820   
2        Fatick                                   45.414538   
3      Kaffrine                                   40.868153   
4       Kaolack                                   45.883738   
5      Kédougou                                   87.544972   
6         Kolda                                   71.850561   
7         Louga                                   23.937548   
8         Matam                                   27.716537   
9   Saint-Louis                                   17.122809   
10      Sédhiou                                   77.410368   
11  Tambacounda                                   55.899698   
12        Thiès                                   22.425229   
13   Ziguinchor                                  100.000000   

    Average Dry Days (<1 mm) Rescaled  Vulnerability R

In [10]:
import pandas as pd
import geopandas as gpd
import xarray as xr
import numpy as np

# Charger les fichiers de données
chirps_path = '/home/laye/Documents/GUIGMA/CHIRPS/merge_chirps-v2.0_1981_2024_p25_Senegal.nc'
shapefile_path = '/home/laye/Documents/GUIGMA/CHIRPS/gadm36_SEN_1.shp'

chirps_data = xr.open_dataset(chirps_path)
regions = gpd.read_file(shapefile_path)

# Indices de vulnérabilité et de capacité d'adaptation
vulnerability_cc_data = {
    'Region': ['Dakar', 'Diourbel', 'Fatick', 'Kaffrine', 'Kaolack', 'Kédougou', 'Kolda', 'Louga', 'Matam', 'Saint-Louis', 'Sédhiou', 'Tambacounda', 'Thiès', 'Ziguinchor'],
    'Vulnerability': [0.9, 1.6, 1.4, 1.4, 1.2, 3.1, 1.5, 3.1, 4.3, 1.2, 1.2, 2.2, 1.3, 0.9],
    'Lack of Coping Capacity': [3.7, 4.3, 5.6, 5.1, 4.6, 6.6, 6.3, 4.6, 5.4, 4.9, 6, 6, 4.3, 5.5]
}

vulnerability_cc_df = pd.DataFrame(vulnerability_cc_data)

results = []

# Liste des années pour lesquelles nous allons extraire les données (sans 2023)
years = list(range(1981, 2023))

# Fonction pour effectuer le rescaling entre 10 et 100
def rescale_to_100(values):
    return (values - np.min(values)) / (np.max(values) - np.min(values)) * 90 + 10

# Itérer sur chaque région
for region_name in regions['NAME_1'].unique():
    region_shape = regions[regions['NAME_1'] == region_name]
    
    # Masquer les données CHIRPS en utilisant la région
    chirps_region = chirps_data.sel(latitude=slice(region_shape.bounds.miny.values[0], region_shape.bounds.maxy.values[0]), 
                                    longitude=slice(region_shape.bounds.minx.values[0], region_shape.bounds.maxx.values[0]))
    
    annual_precipitations = []
    dry_days_list = []

    for year in years:
        # Extraire les données pour l'année spécifiée
        chirps_region_year = chirps_region.sel(time=str(year))
        
        # Calculer la précipitation annuelle totale
        annual_precipitation = chirps_region_year['precip'].sum(dim='time').mean(dim=['latitude', 'longitude']).values
        
        # Calculer la précipitation mensuelle
        monthly_precipitation = chirps_region_year['precip'].groupby('time.month').sum(dim='time').mean(dim=['latitude', 'longitude'])
        
        # Identifier les 3 mois les plus pluvieux
        top3_rainy_months = monthly_precipitation.sortby(monthly_precipitation, ascending=False).month[:3].values
        
        # Extraire les données pour les 3 mois les plus pluvieux
        rainy_months_data = chirps_region_year.sel(time=chirps_region_year['time.month'].isin(top3_rainy_months))
        
        # Calculer le nombre de jours sans précipitations (<1 mm)
        dry_days = (rainy_months_data['precip'] < 1).sum(dim='time').mean(dim=['latitude', 'longitude']).values
        
        # Ajouter aux listes
        annual_precipitations.append(annual_precipitation)
        dry_days_list.append(dry_days)
    
    # Calculer la moyenne des précipitations annuelles et des jours secs
    average_precipitation = np.mean(annual_precipitations)
    average_dry_days = np.mean(dry_days_list)

    # Ajouter les résultats à la liste
    results.append([region_name, average_precipitation.item(), average_dry_days.item()])

# Convertir les résultats en DataFrame
hazard_df = pd.DataFrame(results, columns=['Region', 'Average Annual Precipitation (mm)', 'Average Dry Days (<1 mm)'])

# Rescaling des variables
hazard_df['Average Annual Precipitation (mm) Rescaled'] = rescale_to_100(hazard_df['Average Annual Precipitation (mm)'])
hazard_df['Average Dry Days (<1 mm) Rescaled'] = rescale_to_100(hazard_df['Average Dry Days (<1 mm)'])

# Fusionner les données de hazard avec les indices de vulnérabilité et de capacité d'adaptation
merged_df = pd.merge(hazard_df, vulnerability_cc_df, left_on='Region', right_on='Region')

# Rescaling pour 'Vulnerability' et 'Lack of Coping Capacity'
merged_df['Vulnerability Rescaled'] = rescale_to_100(merged_df['Vulnerability'])
merged_df['Lack of Coping Capacity Rescaled'] = rescale_to_100(merged_df['Lack of Coping Capacity'])

# Créer des fichiers CSV séparés pour chaque variable rééchelonnée
variables = [
    ('Average Annual Precipitation (mm) Rescaled', 'average_annual_precipitation_rescaled.csv'),
    ('Average Dry Days (<1 mm) Rescaled', 'average_dry_days_rescaled.csv'),
    ('Vulnerability Rescaled', 'vulnerability_rescaled.csv'),
    ('Lack of Coping Capacity Rescaled', 'lack_of_coping_capacity_rescaled.csv')
]

# Sauvegarder chaque variable rééchelonnée dans un fichier CSV
for var, filename in variables:
    df = merged_df[['Region', var]]
    output_path = f'/home/laye/Documents/GUIGMA/dry_spell/{filename}'
    df.to_csv(output_path, index=False)

# Afficher les résultats
for var, _ in variables:
    print(merged_df[['Region', var]])


         Region  Average Annual Precipitation (mm) Rescaled
0         Dakar                                   10.000000
1      Diourbel                                   32.871820
2        Fatick                                   45.414538
3      Kaffrine                                   40.868153
4       Kaolack                                   45.883738
5      Kédougou                                   87.544972
6         Kolda                                   71.850561
7         Louga                                   23.937548
8         Matam                                   27.716537
9   Saint-Louis                                   17.122809
10      Sédhiou                                   77.410368
11  Tambacounda                                   55.899698
12        Thiès                                   22.425229
13   Ziguinchor                                  100.000000
         Region  Average Dry Days (<1 mm) Rescaled
0         Dakar                          10.00000

# RISK

In [11]:
import pandas as pd

# Charger les données fusionnées avec les variables rééchelonnées
merged_df = pd.read_csv('/home/laye/Documents/GUIGMA/dry_spell/RISK_rescaled_variables.csv')

# Calculer le HAZARD comme une moyenne des variables rééchelonnées
merged_df['HAZARD'] = (merged_df['Average Annual Precipitation (mm) Rescaled'] +
                       merged_df['Average Dry Days (<1 mm) Rescaled']) / 2

# Calculer le RISK en utilisant la formule : HAZARD * Vulnerability * Lack of Coping Capacity
merged_df['RISK'] = (merged_df['HAZARD'] *
                     merged_df['Vulnerability Rescaled'] *
                     merged_df['Lack of Coping Capacity Rescaled'])

# Sauvegarder les résultats dans un fichier CSV
risk_output_path = '/home/laye/Documents/GUIGMA/dry_spell/RISK_calculated.csv'
merged_df[['Region', 'RISK']].to_csv(risk_output_path, index=False)

# Afficher les résultats
print(merged_df[['Region', 'RISK']])


         Region           RISK
0         Dakar    1000.000000
1      Diourbel   43256.810693
2        Fatick   82794.238146
3      Kaffrine   64802.550791
4       Kaolack   34185.451118
5      Kédougou  388235.259885
6         Kolda  127076.284220
7         Louga  138189.848228
8         Matam  350019.943593
9   Saint-Louis   49634.650994
10      Sédhiou   77651.290185
11  Tambacounda  197009.384168
12        Thiès   18729.291517
13   Ziguinchor   39310.059586


# MAP of the four variables

In [12]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import matplotlib.cm as cm

# Charger les données fusionnées avec les variables rééchelonnées
df = pd.read_csv('/home/laye/Documents/GUIGMA/dry_spell/RISK_rescaled_variables.csv')

# Charger le shapefile des régions du Sénégal
shapefile_path = '/home/laye/Documents/GUIGMA/CHIRPS/gadm36_SEN_1.shp'
regions = gpd.read_file(shapefile_path)

# Fusionner les données avec les informations géographiques
regions = regions.merge(df, left_on='NAME_1', right_on='Region')

# Définir une fonction pour tracer les cartes
def plot_variable(variable, title, output_path):
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    # Normalisation pour avoir la même échelle de couleur pour toutes les variables
    norm = Normalize(vmin=df[variable].min(), vmax=df[variable].max())
    # Tracer la carte
    regions.plot(column=variable, cmap='viridis', linewidth=0.8, ax=ax, edgecolor='0.8', norm=norm, legend=True)
    ax.set_title(title, fontdict={'fontsize': '15', 'fontweight' : '3'})
    ax.set_axis_off()
    # Sauvegarder la carte
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()

# Tracer les cartes pour chaque variable
plot_variable('Average Annual Precipitation (mm) Rescaled', 'Average Annual Precipitation (mm) Rescaled', '/home/laye/Documents/GUIGMA/dry_spell/Annual_Precipitation_Rescaled.png')
plot_variable('Average Dry Days (<1 mm) Rescaled', 'Average Dry Days (<1 mm) Rescaled', '/home/laye/Documents/GUIGMA/dry_spell/Dry_Days_Rescaled.png')
plot_variable('Vulnerability Rescaled', 'Vulnerability Rescaled', '/home/laye/Documents/GUIGMA/dry_spell/Vulnerability_Rescaled.png')
plot_variable('Lack of Coping Capacity Rescaled', 'Lack of Coping Capacity Rescaled', '/home/laye/Documents/GUIGMA/dry_spell/Coping_Capacity_Rescaled.png')

print("Les cartes ont été générées et sauvegardées.")


Les cartes ont été générées et sauvegardées.


In [24]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import matplotlib.ticker as ticker
import numpy as np

# Charger les données fusionnées avec les variables rééchelonnées
df = pd.read_csv('/home/laye/Documents/GUIGMA/dry_spell/RISK_rescaled_variables.csv')

# Charger le shapefile des régions du Sénégal
shapefile_path = '/home/laye/Documents/GUIGMA/CHIRPS/gadm36_SEN_1.shp'
regions = gpd.read_file(shapefile_path)

# Fusionner les données avec les informations géographiques
regions = regions.merge(df, left_on='NAME_1', right_on='Region')

# Définir une fonction pour tracer les cartes avec grille, légende horizontale, et labels
def plot_variable(variable, title, output_path):
    fig, ax = plt.subplots(1, 1, figsize=(12, 12))
    
    # Normalisation pour avoir une échelle de couleur appropriée
    norm = Normalize(vmin=df[variable].min(), vmax=df[variable].max())
    
    # Tracer la carte
    regions.plot(column=variable, cmap='viridis', linewidth=0.8, ax=ax, edgecolor='0.8', norm=norm, legend=True, legend_kwds={'orientation': 'horizontal', 'pad': 0.1, 'shrink': 0.7})
    
    # Ajouter une grille
    ax.grid(True, linestyle='--', linewidth=0.5)
    
    # Définir les ticks pour les coordonnées
    lon_ticks = np.arange(np.floor(regions.total_bounds[0]), np.ceil(regions.total_bounds[2]), 1)
    lat_ticks = np.arange(np.floor(regions.total_bounds[1]), np.ceil(regions.total_bounds[3]), 1)
    ax.set_xticks(lon_ticks)
    ax.set_yticks(lat_ticks)
    
    # Formater les coordonnées
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{x:.2f}'))
    ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda y, _: f'{y:.2f}'))
    
    # Ajouter les labels des régions
    for idx, row in regions.iterrows():
        ax.annotate(text=row['NAME_1'], xy=(row['geometry'].centroid.x, row['geometry'].centroid.y),
                    xytext=(3, 3), textcoords="offset points",
                    fontsize=8, color='black', ha='center', va='center')
    
    # Ajouter le titre
    ax.set_title(title, fontdict={'fontsize': '15', 'fontweight': '3'})
    
    # Ajouter les labels pour les coordonnées sur les axes
    ax.set_xlabel('Longitude', fontsize=12)
    ax.set_ylabel('Latitude', fontsize=12)
    
    # Ajouter les labels des axes pour tous les côtés
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)
    ax.spines['left'].set_visible(True)
    ax.spines['bottom'].set_visible(True)
    
    # Sauvegarder la carte
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()

# Tracer les cartes pour chaque variable
plot_variable('Average Annual Precipitation (mm) Rescaled', 'Average Annual Precipitation (mm) Rescaled', '/home/laye/Documents/GUIGMA/dry_spell/Annual_Precipitation_Rescaled.png')
plot_variable('Average Dry Days (<1 mm) Rescaled', 'Average Dry Days (<1 mm) Rescaled', '/home/laye/Documents/GUIGMA/dry_spell/Dry_Days_Rescaled.png')
plot_variable('Vulnerability Rescaled', 'Vulnerability Rescaled', '/home/laye/Documents/GUIGMA/dry_spell/Vulnerability_Rescaled.png')
plot_variable('Lack of Coping Capacity Rescaled', 'Lack of Coping Capacity Rescaled', '/home/laye/Documents/GUIGMA/dry_spell/Coping_Capacity_Rescaled.png')

print("Les cartes ont été générées et sauvegardées.")


Les cartes ont été générées et sauvegardées.


In [25]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import matplotlib.ticker as ticker
import numpy as np

# Charger les données fusionnées avec les variables rééchelonnées
df = pd.read_csv('/home/laye/Documents/GUIGMA/dry_spell/RISK_rescaled_variables.csv')

# Charger le shapefile des régions du Sénégal
shapefile_path = '/home/laye/Documents/GUIGMA/CHIRPS/gadm36_SEN_1.shp'
regions = gpd.read_file(shapefile_path)

# Fusionner les données avec les informations géographiques
regions = regions.merge(df, left_on='NAME_1', right_on='Region')

# Définir une fonction pour tracer les cartes avec grille, légende horizontale, et labels
def plot_variable(variable, title, output_path):
    fig, ax = plt.subplots(1, 1, figsize=(12, 12))
    
    # Normalisation pour avoir une échelle de couleur appropriée
    norm = Normalize(vmin=df[variable].min(), vmax=df[variable].max())
    
    # Tracer la carte avec le colormap 'Reds'
    regions.plot(column=variable, cmap='Reds', linewidth=0.8, ax=ax, edgecolor='0.8', norm=norm, legend=True, legend_kwds={'orientation': 'horizontal', 'pad': 0.1, 'shrink': 0.7})
    
    # Ajouter une grille
    ax.grid(True, linestyle='--', linewidth=0.5)
    
    # Définir les ticks pour les coordonnées
    lon_ticks = np.arange(np.floor(regions.total_bounds[0]), np.ceil(regions.total_bounds[2]), 1)
    lat_ticks = np.arange(np.floor(regions.total_bounds[1]), np.ceil(regions.total_bounds[3]), 1)
    ax.set_xticks(lon_ticks)
    ax.set_yticks(lat_ticks)
    
    # Formater les coordonnées
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{x:.2f}'))
    ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda y, _: f'{y:.2f}'))
    
    # Ajouter les labels des régions
    for idx, row in regions.iterrows():
        ax.annotate(text=row['NAME_1'], xy=(row['geometry'].centroid.x, row['geometry'].centroid.y),
                    xytext=(3, 3), textcoords="offset points",
                    fontsize=8, color='black', ha='center', va='center')
    
    # Ajouter le titre
    ax.set_title(title, fontdict={'fontsize': '15', 'fontweight': '3'})
    
    # Ajouter les labels pour les coordonnées sur les axes
    ax.set_xlabel('Longitude', fontsize=12)
    ax.set_ylabel('Latitude', fontsize=12)
    
    # Ajouter les labels des axes pour tous les côtés
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)
    ax.spines['left'].set_visible(True)
    ax.spines['bottom'].set_visible(True)
    
    # Sauvegarder la carte
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()

# Tracer les cartes pour chaque variable
plot_variable('Average Annual Precipitation (mm) Rescaled', 'Average Annual Precipitation (mm) Rescaled', '/home/laye/Documents/GUIGMA/dry_spell/Annual_Precipitation_Rescaled.png')
plot_variable('Average Dry Days (<1 mm) Rescaled', 'Average Dry Days (<1 mm) Rescaled', '/home/laye/Documents/GUIGMA/dry_spell/Dry_Days_Rescaled.png')
plot_variable('Vulnerability Rescaled', 'Vulnerability Rescaled', '/home/laye/Documents/GUIGMA/dry_spell/Vulnerability_Rescaled.png')
plot_variable('Lack of Coping Capacity Rescaled', 'Lack of Coping Capacity Rescaled', '/home/laye/Documents/GUIGMA/dry_spell/Coping_Capacity_Rescaled.png')

print("Les cartes ont été générées et sauvegardées.")


Les cartes ont été générées et sauvegardées.


RISK MAPPED

In [13]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import matplotlib.cm as cm

# Charger les données fusionnées avec les variables rééchelonnées et le RISK
df = pd.read_csv('/home/laye/Documents/GUIGMA/dry_spell/RISK_calculated.csv')

# Charger le shapefile des régions du Sénégal
shapefile_path = '/home/laye/Documents/GUIGMA/CHIRPS/gadm36_SEN_1.shp'
regions = gpd.read_file(shapefile_path)

# Fusionner les données avec les informations géographiques
regions = regions.merge(df, left_on='NAME_1', right_on='Region')

# Définir une fonction pour tracer la carte du RISK
def plot_risk():
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    # Normalisation pour avoir une échelle de couleur appropriée
    norm = Normalize(vmin=regions['RISK'].min(), vmax=regions['RISK'].max())
    # Tracer la carte
    regions.plot(column='RISK', cmap='viridis', linewidth=0.8, ax=ax, edgecolor='0.8', norm=norm, legend=True)
    ax.set_title('RISK Map', fontdict={'fontsize': '15', 'fontweight' : '3'})
    ax.set_axis_off()
    # Sauvegarder la carte
    plt.savefig('/home/laye/Documents/GUIGMA/dry_spell/RISK_Map.png', dpi=300, bbox_inches='tight')
    plt.close()

# Tracer la carte du RISK
plot_risk()

print("La carte du RISK a été générée et sauvegardée.")


La carte du RISK a été générée et sauvegardée.


In [19]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import matplotlib.ticker as ticker

# Charger les données fusionnées avec les variables rééchelonnées et le RISK
df = pd.read_csv('/home/laye/Documents/GUIGMA/dry_spell/RISK_calculated.csv')

# Charger le shapefile des régions du Sénégal
shapefile_path = '/home/laye/Documents/GUIGMA/CHIRPS/gadm36_SEN_1.shp'
regions = gpd.read_file(shapefile_path)

# Fusionner les données avec les informations géographiques
regions = regions.merge(df, left_on='NAME_1', right_on='Region')

# Définir une fonction pour tracer la carte du RISK
def plot_risk():
    fig, ax = plt.subplots(1, 1, figsize=(12, 12))
    
    # Normalisation pour avoir une échelle de couleur appropriée
    norm = Normalize(vmin=regions['RISK'].min(), vmax=regions['RISK'].max())
    
    # Tracer la carte
    regions.plot(column='RISK', cmap='viridis', linewidth=0.8, ax=ax, edgecolor='0.8', norm=norm, legend=True, legend_kwds={'orientation': 'horizontal', 'pad': 0.1, 'shrink': 0.7})
    
    # Ajouter le titre
    ax.set_title('RISK Map', fontdict={'fontsize': '15', 'fontweight': '3'})
    
    # Ajouter les labels pour chaque région
    for idx, row in regions.iterrows():
        ax.annotate(text=row['NAME_1'], xy=(row['geometry'].centroid.x, row['geometry'].centroid.y),
                    xytext=(3, 3), textcoords="offset points",
                    fontsize=8, color='black', ha='center', va='center')

    # Ajouter les coordonnées latitude et longitude
    ax.set_xticks(range(int(regions.total_bounds[0]), int(regions.total_bounds[2]) + 1, 1))
    ax.set_yticks(range(int(regions.total_bounds[1]), int(regions.total_bounds[3]) + 1, 1))
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{x:.2f}'))
    ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda y, _: f'{y:.2f}'))
    
    # Ajouter une grille
    ax.grid(True, linestyle='--', linewidth=0.5)
    
    # Sauvegarder la carte
    plt.savefig('/home/laye/Documents/GUIGMA/dry_spell/RISK_Map.png', dpi=300, bbox_inches='tight')
    plt.close()

# Tracer la carte du RISK
plot_risk()

print("La carte du RISK a été générée et sauvegardée.")


La carte du RISK a été générée et sauvegardée.


In [26]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import matplotlib.ticker as ticker

# Charger les données fusionnées avec les variables rééchelonnées et le RISK
df = pd.read_csv('/home/laye/Documents/GUIGMA/dry_spell/RISK_calculated.csv')

# Charger le shapefile des régions du Sénégal
shapefile_path = '/home/laye/Documents/GUIGMA/CHIRPS/gadm36_SEN_1.shp'
regions = gpd.read_file(shapefile_path)

# Fusionner les données avec les informations géographiques
regions = regions.merge(df, left_on='NAME_1', right_on='Region')

# Définir une fonction pour tracer la carte du RISK
def plot_risk():
    fig, ax = plt.subplots(1, 1, figsize=(12, 12))
    
    # Normalisation pour avoir une échelle de couleur appropriée
    norm = Normalize(vmin=regions['RISK'].min(), vmax=regions['RISK'].max())
    
    # Tracer la carte avec le colormap 'Reds'
    regions.plot(column='RISK', cmap='Reds', linewidth=0.8, ax=ax, edgecolor='0.8', norm=norm, legend=True, legend_kwds={'orientation': 'horizontal', 'pad': 0.1, 'shrink': 0.7})
    
    # Ajouter le titre
    ax.set_title('RISK Map', fontdict={'fontsize': '15', 'fontweight': '3'})
    
    # Ajouter les labels pour chaque région
    for idx, row in regions.iterrows():
        ax.annotate(text=row['NAME_1'], xy=(row['geometry'].centroid.x, row['geometry'].centroid.y),
                    xytext=(3, 3), textcoords="offset points",
                    fontsize=8, color='black', ha='center', va='center')
    
    # Ajouter les coordonnées latitude et longitude
    lon_ticks = range(int(regions.total_bounds[0]), int(regions.total_bounds[2]) + 1, 1)
    lat_ticks = range(int(regions.total_bounds[1]), int(regions.total_bounds[3]) + 1, 1)
    ax.set_xticks(lon_ticks)
    ax.set_yticks(lat_ticks)
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{x:.2f}'))
    ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda y, _: f'{y:.2f}'))
    
    # Ajouter une grille
    ax.grid(True, linestyle='--', linewidth=0.5)
    
    # Sauvegarder la carte
    plt.savefig('/home/laye/Documents/GUIGMA/dry_spell/RISK_Map.png', dpi=300, bbox_inches='tight')
    plt.close()

# Tracer la carte du RISK
plot_risk()

print("La carte du RISK a été générée et sauvegardée.")


La carte du RISK a été générée et sauvegardée.
